# ***MAIN APP***

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [3]:
!pip install pyngrok

In [14]:
!ngrok config add-authtoken 2p9ItRHoDqg9qqsryBnioBJq861_tZQ6zXswimnmXv3jmWeF

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import streamlit as st
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

# Load fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/Colab Notebooks/SIH- DOJ CHATBOT/Trained LLM model"  # Path to your fine-tuned model
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Generate a pipeline for text generation
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, pad_token_id=tokenizer.eos_token_id)

def generate_response(query):
    input_text = f"<|startoftext|> {query} |bot|"
    output = generator(input_text, max_length=100, num_return_sequences=1)
    response = output[0]['generated_text'].split("|bot|")[1].split("<|endoftext|>")[0].strip()
    return response

# App Layout
def main():
    if "logged_in" not in st.session_state:
        st.session_state.logged_in = False
    if "conversation" not in st.session_state:
        st.session_state.conversation = []

    # Login Page with Streamlit's native inputs
    if st.session_state.logged_in:
        # Chatbot Page
        st.title("Saul GoodBOT 🤖👨🏻‍⚖")
        st.write("Ask me anything my G!!! 💪😤")

        # Display the ongoing conversation
        for message in st.session_state.conversation:
            if message["role"] == "user":
                # User message
                st.markdown(f'''
                    <div style="border: 2px solid #D7CCC8; border-radius: 10px; padding: 10px; background-color: #D7CCC8; color: #000000; max-width: 80%; text-align: right; margin: 5px;">
                        <strong>You:</strong> {message["text"]}
                    </div>
                ''', unsafe_allow_html=True)
            else:
                # Bot response
                st.markdown(f'''
                    <div style="border: 2px solid #6D4C41; border-radius: 10px; padding: 10px; background-color: #6D4C41; max-width: 80%; text-align: left; margin: 5px;">
                        <strong>Bot:</strong> {message["text"]}
                    </div>
                ''', unsafe_allow_html=True)

        # Input field for user's query
        user_input = st.text_input("Your Query:", key="input_box", help="Type your message here")

        if st.button("Send"):
            if user_input.strip():
                # Add user input to the conversation
                st.session_state.conversation.append({"role": "user", "text": user_input})
                response = generate_response(user_input)
                # Add bot response to the conversation
                st.session_state.conversation.append({"role": "bot", "text": response})

        if st.button("Log Out"):
            st.session_state.logged_in = False
            st.session_state.conversation = []  # Reset conversation on logout
            st.experimental_rerun()

    else:
        # Login Page without custom color
        st.title("Saul GoodBOT 🤖 👨🏻‍⚖ ")
        st.text(" 😮🫵 Do you know that you have rights? The Constitution says you do. 😌☝️ Well you are at right place cause it'Saul goodman!")
        st.text(" ")
        st.text("Fill your accuont details real quick! 📝")

        # Adjusted input fields for username and password using Streamlit's native input
        with st.form(key="login_form"):
            username = st.text_input("Username", key="username_input", max_chars=30)
            password = st.text_input("Password", type="password", key="password_input", max_chars=30)
            submit_button = st.form_submit_button("Login")

        if submit_button:
            if username.strip() and password.strip():  # Ensure both fields are not empty
                st.session_state.logged_in = True
                st.experimental_rerun()  # Refresh after logging in
            else:
                st.error("Please enter both username and password!")

if __name__ == "__main__":
    main()


In [10]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/SIH- DOJ CHATBOT')
!ls

 chatbot.py   DOJ_APP.ipynb	  Model_Training.ipynb	'Trained LLM model'
 Data	     'Main Query Model'   streamlit


In [15]:
!ngrok kill
!ngrok http 8080

ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --url baz.ngrok.dev 8080                           # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --url mydomain.com                              # run ngrok with your own custom domain
  ngrok http 80 --cidr-allow 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions

In [16]:
from subprocess import Popen, PIPE
from pyngrok import ngrok

# Kill any existing tunnels to avoid conflict
ngrok.kill()

# Create a new ngrok tunnel on port 8501 (default Streamlit port)
public_url = ngrok.connect(8501)


print(f"Streamlit is running at: {public_url}")

# Start the Streamlit app (this will use the ngrok tunnel URL to make it accessible publicly)
!streamlit run chatbot.py


Streamlit is running at: NgrokTunnel: "https://e266-34-67-53-107.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.67.53.107:8501

2024-12-03 05:50:46.076137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 05:50:46.105434: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 05:50:46.114160: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 05:50:46.138911: I tensorflow/core/platform/cpu_feature_guard.cc:210] T